In [6]:
#TRANSFER LEARNING ON VGG-16
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers


#from the output above, we saw that block5_pool is the last/top layer of the vgg16 we have so we will add our layers from that point
vgg_model=VGG16(include_top=False, input_shape=(64,64,3))
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])
#for layer in vgg_model.layers:
#  layer.trainable=False

x = layer_dict['block5_pool'].output
#add flatten layer so we can add the fully connected layer later
x = Flatten()(x)
#Fully connected layer
#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.2)(x)
#x = Dense(1024, activation='relu)(x)
#x = Dropout(0.2)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
#this is the final layer so the size of output in this layer is equal to the number of class in our problem
x = Dense(1, activation='sigmoid')(x)
#create the new model
model = Model(inputs=vgg_model.input, outputs=x)
#print(model.summary())

import pandas as pd
dataset=pd.read_csv('/content/drive/My Drive/Colab Notebooks/dogCat/theFinalDataset.csv')
image_array = []
classLabels=[]

for j in range(0, 10000):
 pyString=dataset.iloc[j].tolist()
 classLabels.append(np.array(pyString[0][-1]))
 for index, item in enumerate(pyString):
        data = np.zeros((64,64), dtype=np.uint8)
        pixel_data = item.split()
        for i in range(0, 64):
            pixel_index = i * 64
            data[i] = pixel_data[pixel_index:pixel_index + 64]
        normData=[x/255 for x in data]
        normData=np.asarray(normData)
        normData=np.repeat(normData[:, :, np.newaxis],3,axis=2)
        image_array.append(np.array(normData))
        
        

image_array = np.array(image_array)
classLabels=[int(x) for x in classLabels ]
classLabels=np.asarray(classLabels)

x_train, y_train, x_test, y_test = [], [], [], []
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_array, classLabels, test_size=0.2, random_state=42)

x_train=x_train.reshape(x_train.shape[0],64,64,3)
x_test=x_test.reshape(x_test.shape[0], 64,64,3)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)


gen = ImageDataGenerator(featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)


model.compile(optimizer='adamax', loss="binary_crossentropy", metrics=["accuracy"])

print(y_train)
c_dogs=0
c_cats=0
for i in range(0,len(y_train)):
  if(y_train[i]==0):
    c_dogs=c_dogs+1
  else:
    c_cats=c_cats+1
print("Dogs: ", c_dogs, "\tCats: ", c_cats)

train_generator = gen.flow(x_train, y_train, batch_size=128)
#New Tensorflow update steps_per_epoch = length of x_train divided by the btach size
history=model.fit_generator(train_generator, steps_per_epoch=len(x_train)//128, epochs=37)
#model.fit(x_train, y_train, batch_size=64, epochs=25)
train_score = model.evaluate(x_train, y_train, verbose=1)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

from sklearn.metrics import confusion_matrix
ANSWER=model.predict(x_test)
for i in range(0, len(ANSWER)):
  if(ANSWER[i]>=0.5):
    ANSWER[i]=1
  else:
    ANSWER[i]=0
print(ANSWER)
results=confusion_matrix(y_test, ANSWER)
print(results)

from sklearn.metrics import classification_report 
print(classification_report(y_test, ANSWER))

model_json = model.to_json()
with open("/content/drive/My Drive/Colab Notebooks/dogCat/modelVGG16.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Colab Notebooks/dogCat/modelweightsVGG16.h5")




[0 0 0 ... 1 1 0]
Dogs:  3996 	Cats:  4004


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/37
62/62 [==============================] - 10s 150ms/step - loss: 0.8460 - accuracy: 0.5066
Epoch 2/37
62/62 [==============================] - 9s 151ms/step - loss: 0.6868 - accuracy: 0.5460
Epoch 3/37
62/62 [==============================] - 9s 149ms/step - loss: 0.6706 - accuracy: 0.5844
Epoch 4/37
62/62 [==============================] - 9s 148ms/step - loss: 0.6354 - accuracy: 0.6301
Epoch 5/37
62/62 [==============================] - 9s 148ms/step - loss: 0.5311 - accuracy: 0.7332
Epoch 6/37
62/62 [==============================] - 9s 148ms/step - loss: 0.4269 - accuracy: 0.8074
Epoch 7/37
62/62 [==============================] - 9s 150ms/step - loss: 0.3701 - accuracy: 0.8346
Epoch 8/37
62/62 [==============================] - 9s 151ms/step - loss: 0.3476 - accuracy: 0.8510
Epoch 9/37
62/62 [==============================] - 9s 149ms/step - loss: 0.3422 - accuracy: 0.8470
Epoch 10/37
62/62 [==============================] - 9s 149ms/step - loss: 0.2950 - accuracy: 0.874

In [7]:
from keras.applications import ResNet50
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

#from the output above, we saw that block5_pool is the last/top layer of the vgg16 we have so we will add our layers from that point
resnet50_model=ResNet50(include_top=False, input_shape=(64,64,3))
layer_dict = dict([(layer.name, layer) for layer in resnet50_model.layers])

print(resnet50_model.summary())
x = resnet50_model.output
#add flatten layer so we can add the fully connected layer later
x = Flatten()(x)
#Fully connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
#this is the final layer so the size of output in this layer is equal to the number of class in our problem
x = Dense(1, activation='sigmoid')(x)
#create the new model(recently the Model parameters were changed from input to inputs and output changed to outputs so it's now inputs = resnet50_model.input)
model = Model(inputs=resnet50_model.input, outputs=x)
print(model.summary())

import pandas as pd
dataset=pd.read_csv('/content/drive/My Drive/Colab Notebooks/dogCat/theFinalDataset.csv')
image_array = []
classLabels=[]

for j in range(0, 10000):
 pyString=dataset.iloc[j].tolist()
 classLabels.append(np.array(pyString[0][-1]))
 for index, item in enumerate(pyString):
        data = np.zeros((64,64), dtype=np.uint8)
        pixel_data = item.split()
        for i in range(0, 64):
            pixel_index = i * 64
            data[i] = pixel_data[pixel_index:pixel_index + 64]
        normData=[x for x in data]#Remove the /255 normalization
        normData=np.asarray(normData)
        normData=np.repeat(normData[:, :, np.newaxis],3,axis=2)
        image_array.append(np.array(normData))
        
        

image_array = np.array(image_array)
classLabels=np.asarray(classLabels)
classLabels=[ int(x) for x in classLabels]

x_train, y_train, x_test, y_test = [], [], [], []
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_array, classLabels, test_size=0.2, random_state=42)

x_train=x_train.reshape(x_train.shape[0],64,64,3)
x_test=x_test.reshape(x_test.shape[0], 64,64,3)
#New Tensorflow changes, the y values MUST BE A NUMPY ARRAY can't be a list anymore
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)


gen = ImageDataGenerator(featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
model.compile(optimizer="adamax", loss="binary_crossentropy", metrics=["accuracy"])
train_generator = gen.flow(x_train, y_train, batch_size=256)
history=model.fit_generator(train_generator, steps_per_epoch=len(x_train)//256, epochs=36)
#model.fit(x_train, y_train, batch_size=256, epochs=20)
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

model_json = model.to_json()
with open("/content/drive/My Drive/Colab Notebooks/dogCat/modelResNet50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Colab Notebooks/dogCat/modelweightsResNet50.h5")


94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
__________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/36
31/31 [==============================] - 14s 291ms/step - loss: 1.2401 - accuracy: 0.5891
Epoch 2/36
31/31 [==============================] - 9s 292ms/step - loss: 0.4407 - accuracy: 0.8050
Epoch 3/36
31/31 [==============================] - 9s 291ms/step - loss: 0.3278 - accuracy: 0.8556
Epoch 4/36
31/31 [==============================] - 9s 290ms/step - loss: 0.2581 - accuracy: 0.8921
Epoch 5/36
31/31 [==============================] - 9s 288ms/step - loss: 0.2420 - accuracy: 0.8974
Epoch 6/36
31/31 [==============================] - 9s 288ms/step - loss: 0.1888 - accuracy: 0.9240
Epoch 7/36
31/31 [==============================] - 9s 290ms/step - loss: 0.1758 - accuracy: 0.9296
Epoch 8/36
31/31 [==============================] - 9s 289ms/step - loss: 0.1553 - accuracy: 0.9398
Epoch 9/36
31/31 [==============================] - 9s 289ms/step - loss: 0.1401 - accuracy: 0.9458
Epoch 10/36
31/31 [==============================] - 9s 290ms/step - loss: 0.1227 - accuracy: 0.956